<a href="https://colab.research.google.com/github/harsh-agar/E-Net/blob/master/iNNvestigate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import cv2
from tqdm import tqdm_notebook
import scipy.io as io

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!wget -P '/content/drive/MyDrive/iNNvestigate' -c "https://www.robots.ox.ac.uk/~vgg/data/flowers/102/102flowers.tgz"
!ls '/content/drive/MyDrive/iNNvestigate/'
!tar -xvzf 'r/content/drive/MyDrive/iNNvestigate/102flowers.tgz' -C '/content/drive/MyDrive/iNNvestigate'

In [ ]:
!ls /content/drive/MyDrive/iNNvestigate

102flowers.tgz	dataset  imagelabels.mat  jpg


In [ ]:
label_mat = io.loadmat('/content/drive/MyDrive/iNNvestigate/imagelabels.mat')
len(label_mat['labels'][0])

8189

In [ ]:
img_names = os.listdir('/content/drive/MyDrive/iNNvestigate/jpg/')
img_names.sort()
path = '/content/drive/MyDrive/iNNvestigate/dataset/'
for i in tqdm_notebook(range(len(label_mat['labels'][0]))):
  img = cv2.imread('/content/drive/MyDrive/iNNvestigate/jpg/'+img_names[i])
  if i%5==0:
    folder_path = path+'val/'+str(label_mat['labels'][0][i])
  elif i%5==1:
    folder_path = path+'test/'+str(label_mat['labels'][0][i])
  else:
    folder_path = path+'train/'+str(label_mat['labels'][0][i])
  if not os.path.exists(folder_path):
    os.makedirs(folder_path)
  cv2.imwrite(folder_path+'/'+img_names[i], img) 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


In [3]:
from tensorflow.keras.applications import vgg16
import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D , Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

image_size=224

vgg_conv = vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))

for layer in vgg_conv.layers[:]:
  layer.trainable = False
for layer in vgg_conv.layers:
  print(layer, layer.trainable)

# Create the model
model = Sequential()

# Add the vgg convolutional base model
model.add(vgg_conv)

# Add new layers
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(102, activation='softmax'))

# Show a summary of the model. Check the number of trainable parameters
model.summary()

58892288/58889256 [==============================] - 0s 0us/step
<tensorflow.python.keras.engine.input_layer.InputLayer object at 0x7f2d1e2a4f98> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f2d10fd5eb8> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f2d10fd79e8> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f2cd0578a20> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f2cd0583748> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f2cd0588908> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f2cd0592160> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f2cd0583be0> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f2cd0598f28> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f2cd059dfd0> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f2cd05a3160> Fals

In [4]:
from tensorflow.keras.optimizers import RMSprop
# # Configure the model for training
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(lr=1e-4),
              metrics=['categorical_accuracy'])

In [10]:
# Load the normalized images
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

# Change the batchsize according to your system RAM
train_batchsize = 64
val_batchsize = 64

train_dir = '/content/drive/MyDrive/iNNvestigate/dataset/train'
val_dir = '/content/drive/MyDrive/iNNvestigate/dataset/val'

# Data generator for training data
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(image_size, image_size),
        batch_size=train_batchsize,
        class_mode='categorical')

# Data generator for validation data
validation_generator = validation_datagen.flow_from_directory(
        val_dir,
        target_size=(image_size, image_size),
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=False)

Found 4913 images belonging to 102 classes.
Found 1638 images belonging to 102 classes.


In [11]:
train_generator.samples/train_generator.batch_size

76.765625

In [12]:
import os
fd = os.open(os.devnull, os.O_RDWR)
# NB: even if stdin is closed, fd >= 0
os.dup2(fd, 1)
os.dup2(fd, 2)
if fd > 2:
    os.close(fd)

callbacks = [
        # saving model weights at checkpoints
        keras.callbacks.ModelCheckpoint(
            filepath          = os.path.join('/content/drive/MyDrive/iNNvestigate/models/','otcmodel_{epoch}.h5'),
            save_best_only    = True,
            monitor           = 'val_loss',
            save_weights_only = True,
            verbose           = 1
        )]

# Train the model
history = model.fit(
      train_generator,
      steps_per_epoch=
         train_generator.samples/train_generator.batch_size,
      epochs=20,
      validation_data=validation_generator, 
      validation_steps=
         validation_generator.samples/validation_generator.batch_size,
      verbose=1,
      callbacks=callbacks,
)


Epoch 1/20
76/76 [==============================] - 56s 728ms/step - loss: 4.3779 - categorical_accuracy: 0.1057 - val_loss: 2.8368 - val_categorical_accuracy: 0.3980

Epoch 00001: val_loss improved from inf to 2.83685, saving model to /content/drive/MyDrive/iNNvestigate/models/otcmodel_1.h5
Epoch 2/20
76/76 [==============================] - 47s 613ms/step - loss: 2.5539 - categorical_accuracy: 0.4203 - val_loss: 2.0089 - val_categorical_accuracy: 0.5611

Epoch 00002: val_loss improved from 2.83685 to 2.00888, saving model to /content/drive/MyDrive/iNNvestigate/models/otcmodel_2.h5
Epoch 3/20
76/76 [==============================] - 47s 617ms/step - loss: 1.7097 - categorical_accuracy: 0.5953 - val_loss: 1.5611 - val_categorical_accuracy: 0.6435

Epoch 00003: val_loss improved from 2.00888 to 1.56105, saving model to /content/drive/MyDrive/iNNvestigate/models/otcmodel_3.h5
Epoch 4/20
76/76 [==============================] - 47s 616ms/step - loss: 1.1092 - categorical_accuracy: 0.7438 

In [1]:
pip install --upgrade pillow

     |████████████████████████████████| 2.2MB 17.6MB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: Pillow 7.0.0
    Uninstalling Pillow-7.0.0:
      Successfully uninstalled Pillow-7.0.0


In [5]:
# all_files = os.listdir('/content/drive/MyDrive/iNNvestigate/models/')

all_ckpts = [file for file in os.listdir('/content/drive/MyDrive/iNNvestigate/models/') if ('_' in file) and ('.h5' in file)]
ckpt_epochs = sorted(
    [int(s) for ckpt_name in all_ckpts for s in ckpt_name.replace('_', '.').split('.') if s.isdigit()])
ckpt_epoch = ckpt_epochs[-1]
epoch = ckpt_epoch

print('evaluating epoch %d' % ckpt_epoch)
ckpt = [ckpt for ckpt in all_ckpts if str(ckpt_epoch) in ckpt]
ckpt = ckpt[0]  # maybe otcmodel_5, otcmodel_50, otcmodel_51, ... in ckpt, if epoch = 5
model.load_weights(os.path.join('/content/drive/MyDrive/iNNvestigate/models/', ckpt))


# # Match file name with regular expression and extract epoch number.
# checkpoint_file_regex = re.compile(r'otcmodel_(?P<i>\d+).*\.h5')
# checkpoint_files = {int(m.groupdict()['i']): m.string for f in all_files if (m := re.fullmatch(checkpoint_file_regex, f))}
# # Get latest checkpoint file with best model weights.
# latest_checkpoint = max(checkpoint_files.keys())
# latest_checkpoint_file = checkpoint_files[latest_checkpoint]
# # Load the model weights.
# print_blue(f'Evaluating epoch {latest_checkpoint}')
# model.load_weights(path.join(init_dir, 'model_weights', latest_checkpoint_file))

# # Run Inference.
# pred_vector = model.predict(batched_dataset)  # TODO progress bar


evaluating epoch 20


In [9]:
import os
try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 1.x
    IS_COLAB = True
    if not os.path.exists('/content/drive/MyDrive/iNNvestigate/innvestigate'):
        !git clone https://github.com/albermax/innvestigate.git /content/drive/MyDrive/iNNvestigate/innvestigate
    !pip install /content/drive/MyDrive/iNNvestigate/innvestigate --no-deps
    # %cd /content/innvestigate/examples/notebooks
except Exception:
    IS_COLAB = False

TensorFlow is already loaded. Please restart the runtime to change versions.
Cloning into '/content/drive/MyDrive/iNNvestigate/innvestigate'...
remote: Enumerating objects: 349, done.
remote: Counting objects: 100% (349/349), done.
remote: Compressing objects: 100% (213/213), done.
remote: Total 5666 (delta 245), reused 219 (delta 132), pack-reused 5317
Receiving objects: 100% (5666/5666), 41.31 MiB | 17.72 MiB/s, done.
Resolving deltas: 100% (3970/3970), done.
Processing ./drive/MyDrive/iNNvestigate/innvestigate
  Created wheel for innvestigate: filename=innvestigate-1.0.9-cp36-none-any.whl size=99902 sha256=db4b55184b47aa433c0826153bb0b3a604dfcff0f2278bdd1f68471e9499dd16
  Stored in directory: /tmp/pip-ephem-wheel-cache-0uvhrynt/wheels/ed/6a/81/273f7c945338bda571202a81a08fd5224b3faa94490ba35401
Successfully built innvestigate


In [11]:
!pip install keras==2.3.1
import innvestigate
import innvestigate.utils as iutils
innvestigate.__version__

     |████████████████████████████████| 378kB 13.4MB/s 
     |████████████████████████████████| 51kB 6.7MB/s 
ERROR: innvestigate 1.0.9 has requirement keras==2.2.4, but you'll have keras 2.3.1 which is incompatible.
  Found existing installation: Keras 2.4.3
    Uninstalling Keras-2.4.3:
      Successfully uninstalled Keras-2.4.3


AttributeError: ignored